In [1]:
# Requires brainvisa

In [2]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [4]:
a = anatomist.Anatomist()

global modules:create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
 /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [5]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary"
path_file = "IHI_L_imagen_without_residualization.csv"


In [6]:
with open(json_regions) as f:
    regions = json.load(f)

In [7]:
file_to_display = f"{path_summary}/{path_file}"

In [8]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [9]:
df = pd.read_csv(file_to_display, index_col=0)[["region", "p_value"]]
df["p_value"] = -np.log10(df["p_value"])
res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [10]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("CINGULATE")]

In [11]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [12]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [13]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [14]:
res.head()

,region,p_value,side
2,F.C.L.p.-S.GSM._left,0.185717,left
3,F.C.L.p.-S.GSM._right,0.533102,right
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,2.411015,left
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.455143,right
6,F.C.M.post.-S.p.C._left,1.721546,left


In [15]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,p_value,side,sulcus
2,F.C.L.p.-S.GSM._left,0.185717,left,"[F.C.L.p._left, S.GSM._left]"
3,F.C.L.p.-S.GSM._right,0.533102,right,"[F.C.L.p._right, S.GSM._right]"
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,2.411015,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.455143,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
6,F.C.M.post.-S.p.C._left,1.721546,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"


In [16]:
res = res.sort_values(by="p_value", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,p_value,side,sulcus
46,S.T.i.-S.T.s.-S.T.pol._left,2.951018,left,S.T.i.ant._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.951018,left,S.T.i.post._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.951018,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.951018,left,S.T.pol._left
48,S.T.s.-S.GSM._left,1.890109,left,S.T.s._left
48,S.T.s.-S.GSM._left,1.890109,left,S.GSM._left
47,S.T.i.-S.T.s.-S.T.pol._right,1.589283,right,S.T.i.ant._right
47,S.T.i.-S.T.s.-S.T.pol._right,1.589283,right,S.T.i.post._right
47,S.T.i.-S.T.s.-S.T.pol._right,1.589283,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,1.589283,right,S.T.pol._right


In [17]:
for _, row in res.iterrows():
    print(row.sulcus)

F.Coll._left
S.Rh._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
S.Li.ant._left
S.Li.post._left
F.P.O._left
S.Pa.t._left
S.Cu._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
F.I.P._left
S.Po.C.sup._left
S.Pa.sup._left
F.Coll._right
S.Rh._right
S.s.P._left
S.Pa.int._left
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
F.I.P._left
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._left
OCCIPITAL_left
S.C._left
S.C.sylvian._left
S.Po.C.sup._left
F.I.P.Po.C.inf._left
F.C.L.r.retroC.tr._left
S.T.s._left
S.GSM._left
S.F.inf._left
S.F.inf.ant._left
F.C.L.r.ant._left
F.C.L.r.asc._left
F.C.L.r

In [18]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,p_value,side,sulcus
35,S.F.marginal-S.F.inf.ant._right,0.970395,right,S.F.orbitaire._right


In [19]:
res

,region,p_value,side,sulcus
8,F.Coll.-S.Rh._left,64.931984,left,F.Coll._left
8,F.Coll.-S.Rh._left,64.931984,left,S.Rh._left
56,Sc.Cal.-S.Li._left,49.810711,left,F.Cal.ant.-Sc.Cal._left
56,Sc.Cal.-S.Li._left,49.810711,left,S.O.p._left
56,Sc.Cal.-S.Li._left,49.810711,left,S.Li.ant._left
...,...,...,...,...
21,S.C.-S.Pe.C._right,0.032980,right,S.Pe.C.sup._right
21,S.C.-S.Pe.C._right,0.032980,right,S.Pe.C.median._right
43,S.Po.C._right,0.032381,right,S.Po.C.sup._right
43,S.Po.C._right,0.032381,right,F.I.P.Po.C.inf._right


In [20]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    if row.p_value < -np.log10(0.05/56):
                        vertex['p_value'] = 0.
                    elif vertex['p_value'] != 0.: 
                        vertex['p_value'] = min(vertex['p_value'],
                                                row.p_value)
                    else:
                        vertex['p_value'] = row.p_value
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=-np.log10(0.05/56), maxVal=65,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [21]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
